In [1]:
import sqlite3 as sq3
import pandas.io.sql as pds
import pandas as pd

In [2]:
db_path = "./data/baseball.db"
con = sq3.Connection(db_path)
con

First take a look at the data to see what is available. By querying on `sqlite_master` we get an overview of all available tables in the dataset. Then we can grab the `allstarfull` table and read the data inside.

In [10]:
query = """SELECT * FROM sqlite_master"""
all_tables = pds.read_sql(query, con)
all_tables

,type,name,tbl_name,rootpage,sql
0,table,allstarfull,allstarfull,2,"CREATE TABLE ""allstarfull"" (\n""index"" INTEGER,..."
1,index,ix_allstarfull_index,allstarfull,3,"CREATE INDEX ""ix_allstarfull_index""ON ""allstar..."
2,table,schools,schools,26,"CREATE TABLE ""schools"" (\n""index"" INTEGER,\n ..."
3,index,ix_schools_index,schools,31,"CREATE INDEX ""ix_schools_index""ON ""schools"" (""..."
4,table,batting,batting,99,"CREATE TABLE ""batting"" (\n""index"" INTEGER,\n ..."
5,index,ix_batting_index,batting,100,"CREATE INDEX ""ix_batting_index""ON ""batting"" (""..."


In [6]:
query = """
SELECT * FROM allstarfull
"""
data = pds.read_sql(query, con)
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5148 entries, 0 to 5147
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   index        5148 non-null   int64  
 1   playerID     5148 non-null   object 
 2   yearID       5148 non-null   int64  
 3   gameNum      5148 non-null   int64  
 4   gameID       5099 non-null   object 
 5   teamID       5148 non-null   object 
 6   lgID         5148 non-null   object 
 7   GP           5129 non-null   float64
 8   startingPos  1600 non-null   float64
dtypes: float64(2), int64(3), object(4)
memory usage: 362.1+ KB


,index,playerID,yearID,gameNum,gameID,teamID,lgID,GP,startingPos
0,0,gomezle01,1933,0,ALS193307060,NYA,AL,1.0,1.0
1,1,ferreri01,1933,0,ALS193307060,BOS,AL,1.0,2.0
2,2,gehrilo01,1933,0,ALS193307060,NYA,AL,1.0,3.0
3,3,gehrich01,1933,0,ALS193307060,DET,AL,1.0,4.0
4,4,dykesji01,1933,0,ALS193307060,CHA,AL,1.0,5.0


To find the top 3 players we must group by the player ID. Then we define 'best' by looking a number of games played (`SUM(GP) AS total_GP` and `GROUP BY total_GP DESC`) and starting position (`AVG(startingPosition) AS avg_startingPosition` and `ORDER BY avg_startingPosition ASC`). 

In [9]:
query = """
SELECT playerID, SUM(GP) AS num_games_played, AVG(startingPos) AS avg_starting_pos
FROM allstarfull
GROUP BY playerID
ORDER BY num_games_played DESC, avg_starting_pos ASC
LIMIT 3;
"""
data = pds.read_sql(query, con)
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   playerID          3 non-null      object 
 1   num_games_played  3 non-null      float64
 2   avg_starting_pos  3 non-null      float64
dtypes: float64(2), object(1)
memory usage: 204.0+ bytes


,playerID,num_games_played,avg_starting_pos
0,musiast01,24.0,6.357143
1,mayswi01,24.0,8.000000
2,aaronha01,24.0,8.470588
